In [1]:
import ee
import geemap

In [2]:
# ee.Authenticate()
ee.Initialize()
Map = geemap.Map()

In [3]:
# Define the functions
def i_hate_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

def get_ndvi(image):
    ndvi = image.expression(
        '((NIR - Red) / (NIR + Red))',
        {
            'NIR': image.select('B8'),
            'Red': image.select('B4')
        }
    ).rename('NDVI')
    return ndvi

In [4]:
# Define the Area of Interest (AOI)
aoi_coordinates = [79.42982380063856, 13.53046204298818, 79.43837468297804, 13.538671149113966]
aoi = ee.Geometry.Rectangle(aoi_coordinates)

In [5]:
# Filter Sentinel-2 data and apply functions
dataset = (ee.ImageCollection('COPERNICUS/S2')
           .filterDate('2023-01-01', '2023-12-31')
           .filterBounds(aoi)
           .map(i_hate_clouds)
           .median()
          .clip(aoi))

# Print the resulting dataset
# print(dataset.getInfo())

# Calculate NDVI and mask the result
ndvi = get_ndvi(dataset)
ndvi_mask = ndvi.clip(aoi)

In [6]:
# Visualization parameters for RGB image
rgb_vis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2']
}

# Visualization parameters for NDVI
ndvi_params = {'min': 0, 'max': 0.4, 'palette': ['red', 'yellow', 'green']}

In [7]:
# Add layers to the map
Map.centerObject(aoi, 16)
Map.addLayer(dataset, rgb_vis, 'RGB')
Map.addLayer(ndvi_mask, {'min': 0, 'max': 0.4, 'palette': ['red', 'yellow', 'green']}, 'NDVI')

# Add layer control to the map (this is the correct method)
Map.addLayerControl()

# Display the map
Map

Map(center=[13.534566608764095, 79.43409924180578], controls=(WidgetControl(options=['position', 'transparent_…